In [56]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Load the data

In [57]:
data_train_path = 'C:/Users/Flore.MININT-64PJ14C/Documents/GitHub/ML_Project2/results/Split_48_0.8/Combined_All_Train_90.csv'
tx_train = pd.read_csv(data_train_path,sep=",",squeeze=True)
X_train=tx_train.iloc[:, :48]
ytr = pd.read_csv(data_train_path,sep=",",usecols=[48],squeeze=True)
y_train = tf.keras.utils.to_categorical(ytr,num_classes=14)

In [58]:
print(X_train.shape)
print(y_train.shape)


(10316, 48)
(10316, 14)


# Finding the hyperparameters


In [59]:
from tensorflow.keras.layers import Conv1D, Dropout, MaxPooling1D, Flatten, Dense
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier

In [60]:
n_samples, n_features = X_train.shape[0], X_train.shape[1]
n_outputs=14

In [61]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [53]:
print(X_train.shape)
X_train=np.expand_dims(X_train, axis=-1)
print(X_train.shape)


(10316, 48)
(10316, 48, 1)


## Find best batch size and number of epoch, BEST: bs=100 and epoch=10

In [74]:
def create_model():
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=48, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [75]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
batch_size = [1, 10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.290430 using {'batch_size': 100, 'epochs': 10}
0.212102 (0.039150) with: {'batch_size': 1, 'epochs': 10}
0.209001 (0.038168) with: {'batch_size': 1, 'epochs': 50}
0.209581 (0.044404) with: {'batch_size': 1, 'epochs': 100}
0.262900 (0.056330) with: {'batch_size': 10, 'epochs': 10}
0.221120 (0.055011) with: {'batch_size': 10, 'epochs': 50}
0.219665 (0.052009) with: {'batch_size': 10, 'epochs': 100}
0.269104 (0.057271) with: {'batch_size': 20, 'epochs': 10}
0.228678 (0.034826) with: {'batch_size': 20, 'epochs': 50}
0.213169 (0.033853) with: {'batch_size': 20, 'epochs': 100}
0.285777 (0.054195) with: {'batch_size': 40, 'epochs': 10}
0.235370 (0.060106) with: {'batch_size': 40, 'epochs': 50}
0.209195 (0.049535) with: {'batch_size': 40, 'epochs': 100}
0.270753 (0.061441) with: {'batch_size': 60, 'epochs': 10}
0.231198 (0.028733) with: {'batch_size': 60, 'epochs': 50}
0.225287 (0.048227) with: {'batch_size': 60, 'epochs': 100}
0.281314 (0.025681) with: {'batch_size': 80, 'epochs': 10}

## Find best optimizer, BEST : Adagrad

In [62]:
def create_model(optimizer='Adagrad'):
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=48, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    return model

In [63]:
#create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(optimizer=optimizer)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.277051 using {'optimizer': 'Adagrad'}
0.245549 (0.062457) with: {'optimizer': 'SGD'}
0.215493 (0.037745) with: {'optimizer': 'RMSprop'}
0.277051 (0.045535) with: {'optimizer': 'Adagrad'}
0.076000 (0.008089) with: {'optimizer': 'Adadelta'}
0.232166 (0.025469) with: {'optimizer': 'Adam'}
0.254078 (0.050587) with: {'optimizer': 'Adamax'}
0.242348 (0.050170) with: {'optimizer': 'Nadam'}


## Find best learning rate, BEST : 0.01

In [67]:
def create_model(learn_rate=0.01, momentum=0):
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=48, activation='relu'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Adagrad(lr=learn_rate)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model



In [68]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learn_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.288296 using {'learn_rate': 0.01}
0.278308 (0.057089) with: {'learn_rate': 0.001}
0.288296 (0.045599) with: {'learn_rate': 0.01}
0.225384 (0.048147) with: {'learn_rate': 0.1}
0.226546 (0.040228) with: {'learn_rate': 0.2}
0.208611 (0.027969) with: {'learn_rate': 0.3}


## Find best Network Weight Initialization, BEST: zero

In [70]:
def create_model(init_mode='uniform'):
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(8, input_dim=48, activation='relu',kernel_initializer=init_mode))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Adagrad(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [71]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
param_grid = dict(init_mode=init_mode)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.365164 using {'init_mode': 'zero'}
0.310398 (0.046131) with: {'init_mode': 'uniform'}
0.316896 (0.076015) with: {'init_mode': 'lecun_uniform'}
0.307101 (0.041836) with: {'init_mode': 'normal'}
0.365164 (0.079750) with: {'init_mode': 'zero'}
0.313887 (0.043175) with: {'init_mode': 'glorot_normal'}
0.293625 (0.026594) with: {'init_mode': 'glorot_uniform'}
0.275211 (0.051833) with: {'init_mode': 'he_normal'}
0.312534 (0.072537) with: {'init_mode': 'he_uniform'}


## Find best Neuron Activation Function, BEST: 'relu'

In [81]:
def create_model(activation='relu'):
    # create model
    
    model = tf.keras.Sequential()
    model.add(Dense(6, input_dim=48, activation=activation,kernel_initializer='zero'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Adagrad(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [82]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
param_grid = dict(activation=activation)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.365164 using {'activation': 'relu'}
0.362354 (0.074995) with: {'activation': 'softmax'}
0.346749 (0.059534) with: {'activation': 'softplus'}
0.299345 (0.031852) with: {'activation': 'softsign'}
0.365164 (0.079750) with: {'activation': 'relu'}
0.298569 (0.027330) with: {'activation': 'tanh'}
0.269788 (0.103228) with: {'activation': 'sigmoid'}
0.317088 (0.062008) with: {'activation': 'hard_sigmoid'}
0.286067 (0.046980) with: {'activation': 'linear'}


## Find best Number of Neurons in the Hidden Layer, BEST: 6

In [76]:
def create_model(neurons=6):
    # create model
    model = tf.keras.Sequential()
    model.add(Dense(neurons, input_dim=48, activation='relu',kernel_initializer='zero'))
    model.add(Dense(14, activation='softmax'))
    
    # Compile model
    optimizer = tf.keras.optimizers.Adagrad(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model


In [80]:
# create model
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=100, verbose=0)

# define the grid search parameters
neurons = [6,7 , 8]
param_grid = dict(neurons=neurons)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.365164 using {'neurons': 6}
0.365164 (0.079750) with: {'neurons': 6}
0.365164 (0.079750) with: {'neurons': 7}
0.365164 (0.079750) with: {'neurons': 8}


In [43]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, model):
    epochs, batch_size = 10, 100
    n_samples, n_features = X_train.shape[0], X_train.shape[1]
    n_outputs=14
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size)
    return accuracy

In [44]:
evaluate_model(X_train, y_train, X_test, y_test, model)

Epoch 1/10
8157/9693 [========================>.....] - ETA: 2s - loss: 2.2715 - accuracy: 0.2204

KeyboardInterrupt: 